In [214]:
import numpy as np
from scipy import stats
from scipy.stats import norm

In [ ]:
# ex 1. Crude monte carlo simulator

def confidence_interval(confidence, means, point_estimate):
    std_error = stats.sem(means)
    return stats.t.interval(confidence, df=len(means)-1, loc=point_estimate, scale=std_error)


def crude_monte_carlo(integral_range: tuple, samples: int, func):
    a = integral_range[0]
    b = integral_range[1]
    return func(a, b, samples)

In [221]:
# mean of integral of e^x from 0 to 1
integral_range = (0,1)
samples = 1000000
f = lambda a, b, samples: np.exp(np.random.uniform(a, b, samples))
X = crude_monte_carlo(integral_range, samples, f)

X_point_estimate = np.mean(X)
ci = confidence_interval(0.95, X, X_point_estimate)
print("Exercise 1:")
print("Point estimate:", X_point_estimate)
print(f"Confidence interval: ({ci[0]:.5f}, {ci[1]:.5f})")

Exercise 1:
Point estimate: 1.7191003845599324
Confidence interval: (1.71814, 1.72007)


In [119]:
# Exercise 2
def antithetic_variables(integral_range, samples):
    Ui = np.random.uniform(low=integral_range[0], high=integral_range[1], size=samples)
    Y = (np.exp(Ui) + np.exp(1-Ui)) / 2
    return np.array(Y)

In [220]:
Y = antithetic_variables(integral_range, samples)

Y_point_estimate = np.mean(Y)
ci = confidence_interval(0.95, Y, Y_point_estimate)
print("Exercise 2:")
print("Point estimate:", Y_point_estimate)
print(f"Confidence interval: ({ci[0]:.5f}, {ci[1]:.5f})")

Exercise 2:
Point estimate: 1.7176279216909203
Confidence interval: (1.71371, 1.72155)


In [121]:
# Exercise 3
def control_variate(X, Y, mean_y):
    # Z = X + c * (Y - mu_y)
    c = -np.cov(X, Y)[0][1] / np.var(Y)
    Z = X + c * (Y - mean_y)
    var_Z = np.var(X) - np.cov(X,Y)[0][1] ** 2 / np.var(X)
    return Z, var_Z

In [222]:
# Example from slides
U = np.random.uniform(0, 1, 1000)
Z, var_Z = control_variate(np.exp(U), U, np.mean(U))

Z_point_estimate = np.mean(Z)
print("Exercise 3 with variables from slides:")
print("Z:", Z_point_estimate)
print(f"Var(Z):", var_Z)

Exercise 3 with variables from slides:
Z: 1.695360029209771
Var(Z): 0.1485920324192506


In [223]:
# Exercise 4
def stratified_sampling(U, samples):
    i = np.arange(samples)
    W = np.exp((i + U) / samples).mean(axis=1)
    return W

dimension = int(np.sqrt(samples))
U = np.random.normal(0,1, [dimension, dimension])
W = stratified_sampling(U, dimension)

In [225]:
print("Exercise 4:")
print(f"Stratified sampling mean: {np.mean(W)}")

Exercise 4:
Stratified sampling mean: 1.7174249172468627


In [163]:
# Exercise 5
W, var_W = control_variate(W, np.random.uniform(0, 1, int(np.sqrt(samples))), 1/2)

print('Exercise 5: ')
print("W:", np.mean(W))
print("Var(W):", var_W)

Exercise 5: 
W: 1.7174263430809455
Var(W): -0.00035022083237356573


In [ ]:
# Exercise 6
def stratified_sampling_with_cv(U, samples):
    i = np.arange(samples)[:,None]
    x = (i + U) / samples

    f = np.exp(x)
    g = x

    g_mean = (i + 1/2) / samples

    cov_fg = np.array([np.cov(f[k], g[k])[0, 1] for k in range(samples)])
    var_g = np.var(g, axis=1)
    c = -cov_fg / var_g
    
    Z = f + c[:, None] * (g - g_mean)
    
    stratum_estimates = Z.mean(axis=1)
    stratum_variances = np.var(Z, axis=1, ddof=1) / U.shape[1]
    
    integral_estimate = stratum_estimates.sum() / samples
    total_variance = stratum_variances.sum() / (samples ** 2)
    
    return integral_estimate, total_variance


In [ ]:
K = 10 
n_k = 100
U = np.random.random((K, n_k))

estimate, variance = stratified_sampling_with_cv(U, K)

print('Exercise 5')
print(f"Estimate: {estimate:.6f} (True = {np.exp(1)-1:.6f})")
print(f"Variance: {variance:.3e}")

Estimate: 1.718258 (True = 1.718282)
Variance: 7.006e-10


In [209]:
# Exercise 7
f = lambda a, b, samples: np.random.normal(a, b, samples)
a = 2
sigma_2 = 1
samples = 1000

def crude_monte_carlo(integral_range: tuple, samples: int, func):
    a = integral_range[0]
    b = integral_range[1]
    return func(a, b, samples)

Z = crude_monte_carlo((0,1), 1000, f)

In [212]:
# Exercise 7
print("Odds of Z > a:", (Z>a).mean())
print("Variance:", (X>a).var() / samples)

Odds of Z > a: 0.025
Variance: 2.4374999999999996e-05


In [218]:
def importance_sampling(a, n_samples=10000, shift_mean=a, shift_var=1):
    """Importance sampling estimator for P(Z > a)"""
    # Sample from shifted normal
    Y = np.random.normal(loc=shift_mean, scale=np.sqrt(shift_var), size=n_samples)
    # Calculate importance weights
    f = norm.pdf(Y)  # Standard normal PDF
    g = norm.pdf(Y, loc=shift_mean, scale=np.sqrt(shift_var))  # Proposal PDF
    h = (Y > a).astype(float)
    W = h * f / g
    return np.mean(W), np.var(W)/n_samples

# Experiment for different a values
for a in [2, 4]:
    print(f"\nFor a = {a}:")
    # Crude MC
    f = lambda mu, sigma, samples: np.random.normal(mu, sigma, samples)
    Z = crude_monte_carlo((0,1), 10000, f)
    est = (Z>a).mean()
    var = (Z>a).var() / 10000
    print(f"Crude MC: P(Z>{a}) = {est:.6f}, Var = {var:.2e}")
    
    # Importance sampling
    est_is, var_is = importance_sampling(a)
    print(f"Importance Sampling: P(Z>{a}) = {est_is:.6f}, Var = {var_is:.2e}")
    print(f"Variance reduction: {100*(var-var_is)/var:.1f}%")


For a = 2:
Crude MC: P(Z>2) = 0.023100, Var = 2.26e-06
Importance Sampling: P(Z>2) = 0.022592, Var = 8.40e-07
Variance reduction: 62.8%

For a = 4:
Crude MC: P(Z>4) = 0.000000, Var = 0.00e+00
Importance Sampling: P(Z>4) = 0.000032, Var = 4.66e-13
Variance reduction: -inf%


/var/folders/rl/wmw8y4kx7nl7flgwk7c8y81h0000gp/T/ipykernel_70952/2707125559.py:25: RuntimeWarning: divide by zero encountered in scalar divide
  print(f"Variance reduction: {100*(var-var_is)/var:.1f}%")


In [219]:
# Exercise 8
def importance_sampling_exp(n_samples=10000, lambd=1):
    """Importance sampling with exponential proposal"""
    # Sample from exponential
    Y = np.random.exponential(scale=1/lambd, size=n_samples)
    # Only keep samples in [0,1]
    Y = Y[Y <= 1]
    if len(Y) == 0:
        return 0, 0  # Edge case when lambda is too large
    
    # Calculate weights
    f = 1  # Uniform PDF
    g = lambd * np.exp(-lambd * Y)  # Exponential PDF
    h = np.exp(Y)
    W = h * f / g
    
    # Correct for truncation to [0,1]
    correction = 1 - np.exp(-lambd)  # P(Y ≤ 1)
    return np.mean(W) * correction, np.var(W)/len(Y) * correction**2

# Find optimal lambda by minimizing variance
lambdas = np.linspace(0.1, 10, 50)
variances = []
for l in lambdas:
    _, var = importance_sampling_exp(lambd=l)
    variances.append(var)
    
optimal_lambda = lambdas[np.argmin(variances)]
print(f"\nOptimal lambda: {optimal_lambda:.2f}")

# Compare with true integral (e^1 - e^0 ≈ 1.71828)
est, var = importance_sampling_exp(lambd=optimal_lambda)
print(f"IS estimate: {est:.6f}, True value: {np.exp(1)-1:.6f}")
print(f"Variance: {var:.2e}")


Optimal lambda: 0.50
IS estimate: 1.706005, True value: 1.718282
Variance: 1.45e-04
